In [1]:
import sys
sys.path.insert(0, "/home/uih/JYL/Programs/YOLO/")
import torch
from models import RetinaNet

In [2]:
class_num = 20

In [3]:
model = RetinaNet(9, class_num, [3, 4, 6, 3], freeze_bn=False)
my_state_dict = model.state_dict()

In [4]:
my_total_k = []
my_total_v = []
for i, (k, v) in enumerate(my_state_dict.items()):
    my_total_k.append(k)
    my_total_v.append(v)
    # print(f'{i}: {k}\t\t{v.shape}')

In [5]:
org_state_dict = torch.load('/home/uih/JYL/Programs/YOLO_ckpts/retinanet_for_coco.pth', map_location='cpu')

In [6]:
org_state_dict = org_state_dict['model_state_dict']

In [7]:
org_total_k, org_total_v = [], []

for i, (k, v) in enumerate(org_state_dict.items()):
    org_total_k.append(k)
    org_total_v.append(v)
    # print(f'{i}: {k}\t\t{v.shape}')

In [8]:
i = 0
for org, my in zip(org_total_v, my_total_v):
    if org.shape != my.shape:
        print(i)
    i += 1

342
343


## load prarmeters

In [9]:
i = 0
total = org_total_v[:]
for m in model.modules():
    if i in [342, 343]:
        i += 1
        continue
    if isinstance(m, torch.nn.Conv2d):
        assert m.weight.data.shape == total[i].shape, f"{m.weight.data.shape},\t{total[i].shape}"
        m.weight.data = total[i]
        # print(m.weight.data.shape, total[i].shape)
        i += 1
        if m.bias is not None:
            assert m.bias.data.shape == total[i].shape
            m.bias.data = total[i]
            i += 1
        
    if isinstance(m, torch.nn.BatchNorm2d):
        assert m.weight.data.shape == total[i].shape
        m.weight.data = total[i]
        i += 1
        assert m.bias.data.shape == total[i].shape
        m.bias.data = total[i]
        i += 1
        assert m.running_mean.shape == total[i].shape
        m.running_mean = total[i]
        i += 1
        assert m.running_var.shape == total[i].shape
        m.running_var = total[i]
        i += 1
        m.num_batches_tracked = total[i]
        i += 1

In [10]:
i

354

## check

In [11]:
i = 0
total = org_total_v[:]
for m in model.modules():
    if i in [342, 343]:
        i += 1
        continue
    if isinstance(m, torch.nn.Conv2d):
        assert (m.weight.data == total[i]).all()
        i += 1
        if m.bias is not None:
            assert (m.bias.data == total[i]).all()
            i += 1
    if isinstance(m, torch.nn.BatchNorm2d):
        assert (m.weight.data == total[i]).all()
        i += 1
        assert (m.bias.data == total[i]).all()
        i += 1
        assert (m.running_mean == total[i]).all()
        i += 1
        assert (m.running_var == total[i]).all()
        i += 1
        assert (m.num_batches_tracked == total[i]).all()
        i += 1

In [12]:
my_output_state = model.cpu().state_dict()
torch.save({'model_state_dict': my_output_state}, "/home/uih/JYL/Programs/YOLO_ckpts/retinanet_for_voc.pth")

In [14]:
st = torch.load('/home/uih/JYL/Programs/YOLO_ckpts/retinanet_for_voc.pth')
model.load_state_dict(st["model_state_dict"])

<All keys matched successfully>